<center><img src="images/MLU-NEW-logo.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>


# <a name="0">Improve Factual Consistency Part 2 </a>
## <a name="0">Improving Factual Consistency and Explainability using reasoning via LLM Consultancy </a>

### Glossary of Terms
- Naive Judge : This LLM has **no** access to transcript but only question and two summaries. Measure the baseline performance.
- Expert Judge : This LLM has access to transcript along with question and two summaries
- Question asked to LLM (in all experiments): It is always the same: `Which one of these summaries is the most factually consistent one?`

## Dataset
Our dataset is distilled from the Amazon Science evaluation benchmark dataset called <a href="https://github.com/amazon-science/tofueval">TofuEval</a>. 10 summaries have been curated from the [MediaSum documents](https://github.com/zcgzcgzcg1/MediaSum) inside the tofueval dataset for this notebook. 

MediaSum is a large-scale media interview dataset contains 463.6K transcripts with abstractive summaries, collected from interview transcripts and overview / topic descriptions from NPR and CNN.

## LLM Access

We will need access to Anthropic Claude v3 Sonnet, Mistral 7b and  Mixtral 8x7b LLMs for this notebook.

[Anthropic Claude v3(Sonnet)](https://www.anthropic.com/news/claude-3-family) , [Mixtral 8X7B](https://mistral.ai/news/mixtral-of-experts/), [Mistral 7B](https://mistral.ai/news/announcing-mistral-7b/) - all of them pre-trained on general text summarization tasks.

## Notebook Overview

In this notebook, we navigate the LLM debating technique with more persuasive LLMs having two expert debater LLMs (Claude and Mixtral) and one judge (using Claude - we can use others like Mistral/Mixtral, Titan Premier) to measure, compare and contrast its performance against other techniques like self-consistency (with naive and expert judges) and LLM consultancy. This notebook is an adapted and partial implementation of one of the ICML 2024 best papers, <a href="https://arxiv.org/pdf/2402.06782"> Debating with More Persuasive LLMs Leads to More Truthful Answers </a> on a new and different Amazon Science evaluation dataset <a href="https://github.com/amazon-science/tofueval">TofuEval</a>. 


- Part 1.  Demonstrate typical Standalone LLM approach

- Part 2.  **[THIS notebook]** Demonstrate the LLM Consultancy approach and compare with Part 1.

- Part 3.  Demonstrate the LLM Debate approach and compare with other methods.



<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    While this notebook(part 1,2 and 3) compares various methods and demonstrates the efficacy of LLM Debates in notebook part 3 with a supervised dataset, the greater benefit is possible in unsupervised scenarios where ground truth is unknown and ground truth alignment and/or curation is required. Human annotation can be expensive plus slow and agreement amongst human annotators adds another level of intricacy. A possible `scalable oversight direction could be this LLM debating technique to align on the ground truth options` via this debating and critique mechanism by establishing factual consistency(veracity). This alignment and curation of ground truth for unsupervised data could be a possible win direction for the debating technique in terms of cost versus benefit analysis.
</div>
<br/>


#### Notebook Kernel
Please choose `conda_python3` as the kernel type of the top right corner of the notebook if that does not appear by default.

#### LLMs used
[Anthropic Claude v3(Sonnet)](https://www.anthropic.com/news/claude-3-family) , [Mixtral 8X7B](https://mistral.ai/news/mixtral-of-experts/), [Mistral 7B](https://mistral.ai/news/announcing-mistral-7b/) - all of them pre-trained on general text summarization tasks.

## Use-Case Overview

To demonstrate the measurement and improvement of factual consistency (veracity) with explainability in this notebook, we conduct a series of experiments to choose the best summary for each transcript. In each experiment, we measure the veracity and correctness of the summaries generated from transcripts and improve upon the decision to choose the correct one via methods like LLM consultancy and LLM debates.

The <b>overall task in this notebook</b> is choose which one of the two summaries is most appropriate for a given transcript. There are a total of 10 transcripts and each transcript has 2 summaries - one correct and other incorrect. The incorrect summaries have various classes of errors like `Nuanced Meaning Shift`, `Extrinsic Information` and  `Reasoning errors`. 

In this notebook we will conduct the following set of experiment combinations to measure, compare and contrast LLM debating techniques with others.

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    If you see throttling exception, please increase timeout from 10 seconds in `time.sleep(10)` to say 20 and retry
</div>
<br/>

## Experiments
For each of these experiments we flip the side of the argument the LLM takes to account for `position bias` and `verbosity bias` and re-run each experiment.

**Note** We always use the same Judge LLM (Mistral 7B) across all the experiments in this notebook



---

### Experiment 3: (LLM consultancy) 
<center><img src="images/veracitylab01-llm-consultancy.png" alt="In this image, we depict the flow of LLM Consulancy. First a consultant LLMs is assigned a side to defend.They persuade the judge why their choice of summary is correct
based on transcript contents. Next each consultation from the LLM is saved to a file and the consultant picks up the entire rationale history before posting their next thought. Finally, Once all 3 rounds of consultancy are over, the Judge LLM reads all the content and decides whether to agree or disagree with the consultant."  height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>
We use Claude as consultancy for both sides of the answers separately and then take the average of both the experiments 3a and 3b as final accuracy. This continues for N(=3 in this notebook) rounds. This accounts for errors due to position (choosing an answer due to its order/position) and verbosity bias (one answer longer than the other)

##### Experiment 3a: (LLM consultancy for Answer A): 
Claude v3(Sonnet) acting as a consultant always picks Answer A(Ground Truth:False Answer) and shares rationale why that answer is correct. This continues for N(=3 in this notebook) rounds. At the end of these rounds, Claude as a judge adjudicates whether Claude as a debater's rationale is correct and if answer A is correct or not.
##### Experiment 3b: (LLM consultancy for Answer B): 
Claude v3(Sonnet) acting as a consultant always picks Answer B(Ground Truth:True Answer) and generates rationale why that answer is correct. This continues for N(=3 in this notebook) rounds. At the end of these rounds, Claude  as a judge adjudicates whether Claude as a debater rationale is correct and if answer B is correct or not.

---

---
## Evaluation Metrics
For each type of experiment we evaluate the accuracy of the answers for that experiment/method type to compare and contrast each method at the end.

For the final experiment on LLM Debate, we also calculate the `win rate` of the LLM debaters to evaluate which of the LLMs actually got most of the answers right as adjudicated by the judge. This can be considered a mechanism to choose one LLM over the other given this use-case.

---


This notebook notebook has the following sections:

1. <a href="#1">Dataset exploration</a>
8. <a href="#8">LLM Consultancy: 1 expert LLM consulting for 2nd summary , 1 naive judge</a>
9. <a href="#9">LLM Consultancy: 1 expert LLM consulting for 1st summary, 1 naive judge</a>
10. <a href="#10">Accuracy of LLM Consultancy</a>
14. <a href="#14">Compare Accuracies across experiments</a>
16. <a href="#16">Challenge exercise</a>
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.

---

In [1]:
%%capture
!pip3 install setuptools==70.0.0

In [2]:
%%capture
!pip3 install -q -U pip --root-user-action=ignore
!pip3 install -q -r requirements.txt --root-user-action=ignore

In [ ]:
# We load all prompts from a separate file prompts.py
%load_ext autoreload
%autoreload 2
from prompts import *

%load_ext autoreload
%autoreload 2
from mlu_utils.veracity_utils import *

In [ ]:
clean_up_files_in_dir("./transcripts")
clear_file_contents("./log_files/notebook_run_logs.log")

In [5]:
import boto3
import re, time
import random
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

from langchain.llms.bedrock import Bedrock
from langchain.prompts import PromptTemplate
from IPython.display import Markdown
from collections import Counter

from IPython.display import Markdown, display
import logging
import boto3, warnings
import pandas as pd
# Supress warnings
warnings.filterwarnings("ignore")
logging.basicConfig(filename='log_files/notebook_run_logs.log', encoding='utf-8', level=logging.INFO)
logger = logging.getLogger(__name__)
logger.info("----- Test logging setup -----")


### Bedrock Model Access check

In [ ]:
#test if all bedrock model access has been enabled 
test_llm_calls()

### Constants used in this notebook

In [7]:
number_of_rounds = 3
question = "Which one of these summaries is the most factually consistent one?"
total_data_points = 10

### <a name="1">Dataset Exploration</a>
(<a href="#0">Go to top</a>)


In [ ]:
# pre-process the dataset
answers_df = pd.read_csv("./tofueval_dataset/mediasum_dev_doc_id_group_final_dual_summaries_manual_final_dataset.csv")
#answers_df.head()
interview_df = pd.read_csv("./tofueval_dataset/mediasum_dev_doc_complete_final.csv")
#interview_df.head()

result = pd.merge(answers_df, interview_df, on="doc_id")
final_dataset = result[["doc_id", "topic", "summ_sent_incorrect_original", "summ_sent_correct_manual", "exp", "type", "source"]]
final_dataset

### <a name="8">LLM Consultancy: 1 expert LLM consulting for 2nd summary , 1 naive judge</a>
(<a href="#0">Go to top</a>)

LLM Consultancy with Claude (defending False) as Consultant for Answer B (incorrect) and Claude as Judge for 3 rounds.

In this experiment we choose Claude v3(Sonnet) to act as a LLM consultant always picking Answer B(Ground Truth:False Answer). Claude also outputs its rationale why that answer is correct. This continues for N(=3 in this notebook) rounds. At the end of these rounds, Claude as a judge adjudicates whether Claude's rationale is correct and if answer A is correct or not.

In [ ]:
%%time

for index, row in final_dataset.iterrows():
    time.sleep(10) # avoid throttling exceptions
    debate_id = row['doc_id']
    answer_a = row['summ_sent_correct_manual']
    answer_b = row['summ_sent_incorrect_original']
    complete_interview_transcript = row['source']
    claude_defending_summary=answer_b
    claude_opposing_summary=answer_a
    delete_file(f"./transcripts/full_transcript_consultancy_{debate_id}.log")

    logger.info(f"-------------Consultancy Debate_id {debate_id}-------------------")
    for round_number in range(number_of_rounds):
        time.sleep(10) # avoid throttling exceptions
        logger.info(f"START Consultancy with Claude Debate_id {debate_id} Round #{round_number + 1} >>>>>> \n")

        consultancy_response = invoke_claude_v3(debate_id = debate_id, 
                         round_number = round_number + 1,
                         question=question,
                         summary_defending = claude_defending_summary, 
                         summary_opposing = claude_opposing_summary, 
                         complete_interview = complete_interview_transcript,
                         debate=False
                         )
        logger.info(f"END Consultancy with Claude Debate_id {debate_id} Round #{round_number + 1} >>>>>> \n")
    print(f"=========== Consultancy debate_id {debate_id} Round #1..{round_number + 1} ======= saved in ./transcripts/full_transcript_consultancy_{debate_id}.log")
    


### Judge decision for LLM Consulting on 2nd summary

In [ ]:
%%time
consultancy_judge_regular_answers = list()
for index, row in final_dataset.iterrows():
    time.sleep(10) # avoid throttling exceptions
    debate_id = row['doc_id']
    answer_a = row['summ_sent_correct_manual']
    answer_b = row['summ_sent_incorrect_original']
    complete_interview_transcript = row['source']
    logger.info(f"-------------CONSULTANCY JUDGE Debate_id {debate_id}-------------------")


    judge_response = invoke_claude_judge_consultant(debate_id = debate_id,
                                  question = question, 
                                  answer_a = answer_a,
                                  answer_b = answer_b)
    consultancy_judge_regular_answers.append(extract_final_answer(judge_response, flipped=False))
    logger.info(f" >>>>> invoke_titan_judge_consultant - judge_response  >>>>> {judge_response}")
    # Print the final response 
    format_final_response(debate_id, 
                          round_num=1, 
                          question=question, 
                          answer_a=answer_a, 
                          answer_b=answer_b, 
                          judge_response=judge_response)
    


### <a name="9">LLM Consultancy: 1 expert LLM consulting for 1st summary, 1 naive judge</a>
(<a href="#0">Go to top</a>)

**FLIPPED LLM CONSULTANCY** with Claude (defending True) as Consultant and Titan as Judge:


In this experiment we choose Claude v3(Sonnet) to act as a LLM consultant always picking Answer A(Ground Truth:True Answer). Claude also outputs its rationale why that answer is correct. This continues for N(=3 in this notebook) rounds. At the end of these rounds, Claude as a judge adjudicates whether Claude's rationale is correct and if answer A is correct or not.

In [ ]:
%%time

for index, row in final_dataset.iterrows():
    time.sleep(10) # avoid throttling exceptions
    debate_id = row['doc_id']
    answer_a = row['summ_sent_correct_manual']
    answer_b = row['summ_sent_incorrect_original']
    complete_interview_transcript = row['source']
    claude_defending_summary=answer_a
    claude_opposing_summary=answer_b
    
    logger.info(f"-------------Consultancy Flipped Debate_id {debate_id}-------------------")

    #### Consultancy Claude - defending true - 3 rounds
    delete_file(f"./transcripts/full_transcript_consultancy_{debate_id}{FLIPPED_FILE_SUFFIX}.log")
    
    for round_number in range(number_of_rounds):
        time.sleep(10) # avoid throttling exceptions
        logger.info(f"START Flipped Consultancy with Claude Round #{round_number + 1} >>>>>> \n")

        consultancy_response = invoke_claude_v3(debate_id = debate_id + FLIPPED_FILE_SUFFIX, 
                         round_number = round_number + 1,
                         question=question,
                         summary_defending = claude_defending_summary, 
                         summary_opposing = claude_opposing_summary, 
                         complete_interview = complete_interview_transcript,
                         debate=False
                         )
        logger.info(f" >>>>> consultancy_response Round #{round_number + 1} >>>>> {consultancy_response}")
        logger.info(f"END Flipped Consultancy with Claude debate_id {debate_id}  Round #{round_number + 1} >>>>>> \n")
    print(f"=========== END OF Flipped Consultancy debate_id {debate_id} Round #1..{round_number + 1} ======= \n")
    

In [ ]:
%%time


consultancy_judge_flipped_answers = list()
for index, row in final_dataset.iterrows():
    time.sleep(10) # avoid throttling exceptions
    debate_id = row['doc_id']
    answer_a = row['summ_sent_correct_manual']
    answer_b = row['summ_sent_incorrect_original']
    complete_interview_transcript = row['source']
    logger.info(f"-------------CONSULTANCY Flipped JUDGE Debate_id {debate_id}-------------------")

    time.sleep(4) # sleep 4 seconds to fix timeout errors
    judge_response = invoke_claude_judge_consultant(debate_id = debate_id + FLIPPED_FILE_SUFFIX,
                                  question = question, 
                                  answer_a = answer_a,
                                  answer_b = answer_b)

    logger.info(f" >>>>> Flipped invoke_titan_judge_consultant - judge_response  >>>>> {judge_response}")
    consultancy_judge_flipped_answers.append(extract_final_answer(judge_response, flipped=False))
    
    # Print the final response 
    format_final_response(debate_id, 
                          round_num=1, 
                          question=question, 
                          answer_a=answer_a, 
                          answer_b=answer_b, 
                          judge_response=judge_response)

### <a name="8">Accuracy of LLM Consultancy</a>
(<a href="#0">Go to top</a>)

In [ ]:
consultancy_judge_regular_answers

In [ ]:
consultancy_judge_flipped_answers

In [15]:
accuracy_consultant_judge = find_num_matching_elements(consultancy_judge_regular_answers, consultancy_judge_flipped_answers)/total_data_points

In [ ]:
accuracy_consultant_judge

In [ ]:
# save the results
results_dict = {"accuracy_consultant_judge" : accuracy_consultant_judge}
save_each_experiment_result(results_dict)
print("notebook results saved in results folder")

## <a name="14">Compare Accuracies across experiments/methods.</a>
(<a href="#0">Go to top</a>)

Here we compare the accuracies of each method/experiment to understand

In [ ]:
accuracy_naive_judge = get_each_experiment_result("accuracy_naive_judge")
accuracy_expert_judge = get_each_experiment_result("accuracy_expert_judge")

final_accuracy_comparison_judge_and_consultant(
    accuracy_naive_judge = accuracy_naive_judge,
    accuracy_expert_judge = accuracy_expert_judge,
    accuracy_consultant_judge = accuracy_consultant_judge
)

In [ ]:
# Build the plot
%matplotlib inline
x_values = [ "Naive Judge", "Expert Judge", "LLM Consultant"]
y_values = [ accuracy_naive_judge, accuracy_expert_judge, accuracy_consultant_judge]
plt.bar(x_values, y_values)
plt.title('Compare Accuracies across experiments')
plt.xlabel('Experiment Type')
plt.ylabel('Accuracy')
 
plt.show()